In [1]:
import pandas as pd
import csv
import sqlite3

In [2]:
cd "C:/Users/drewh/Documents/Flatiron Documents/Phase 1/movie_studio_analysis/zippedData"

C:\Users\drewh\Documents\Flatiron Documents\Phase 1\movie_studio_analysis\zippedData


CSV and STV:

    bom.movie_gross.csv.gz
    3386 rows, 5 columns
    rows - individual films
    Columns - title, studio, domestic gross, foreign gross, year
    
    tmdb.movies.csv.gz
    26517 rows, 10 columns
    Rows - individual films
    Columns - Unnamed: 0 (index duplicate), genre_ids, id, original_language, original_title, popularity, release_date, title, vote_average, vote_count
    
    tn.movie_budgets.csv.gz
    5782 rows, 6 columns
    Rows - individual films
    Columns - id, release_date, movie, production_budget, domestic_gross, worldwide_gross
    
    rt.movie_info.tsv.gz
    To open: pd.read_csv("./rt.movie_info.tsv.gz", sep="\t")
    1560 rows, 12 columns
    Rows - individual films
    Columns - id, synopsis, rating, genre, director, writer, theater_date, dvd_date, currency, box_office, runtime, studio
    
    ./rt.reviews.tsv.gz
    to open: pd.read_csv("./rt.reviews.tsv.gz", sep="\t", encoding = 'unicode_escape')
    54432 rows, 8 columns
    Rows - 
    Columns - id, review, rating, fresh, critic, top_critic, publisher, data

SQL Tables

    movie_basics
    146144 rows, 6 columns
    Columns: movie_id, primary_title, original_title, start_year,
             runtime_minutes, genres
    
    directors
    291174 rows, 2 columns
    Columns: movie_id, person_id
    
    known_for
    1638260 rows, 2 columns
    Columns: person_id, movie_id
    
    movie_akas
    331703 rows, 8 columns
    Columns: movie_id, ordering, title, region, language, types, attributes, is_original_title
    
    movie_ratings
    73856 rows, 3 columns
    Columns: movie_id, averagerating, numvotes
    
    persons
    606648 rows × 5 columns
    Columns: person_id, primary_name, birth_year, death_year, primary_profession
             
    principals
    1028186 rows, 6 columns
    Columns: movie_id, ordering, person_id, category, job, characters
    
    writers
    255873 rows, 2 columns
    Columns: movie_id, person_id

Synthesis!
We have two tables with income information: BOM and TN. There's also some in the RT TSV, but very few of those films have box office values associated with them.

Variables we have:
-Runtime
-Studio
-Rating
-Genre
-Original language
-Production budget
-Ratings(i.e. reviews)
-People
    -Writers, directors, actors
    
For audience: the money made is the measure of success. What impacts that?
    -Ratings is an intermediary. Do positive ratings help? What improves ratings?
    -Are there particularly profitable individuals? Writers, directors, actors, etc.?

In [3]:
ls

 Volume in drive C is Windows-SSD
 Volume Serial Number is 5CA0-02C2

 Directory of C:\Users\drewh\Documents\Flatiron Documents\Phase 1\movie_studio_analysis\zippedData

10/03/2022  03:43 PM    <DIR>          .
10/03/2022  10:42 AM    <DIR>          ..
09/30/2022  05:14 PM            53,544 bom.movie_gross.csv.gz
12/20/2021  04:31 PM       169,443,328 Ignore - im.db
10/03/2022  03:43 PM                 0 im.db
09/30/2022  05:14 PM           498,202 rt.movie_info.tsv.gz
09/30/2022  05:14 PM         3,402,194 rt.reviews.tsv.gz
09/30/2022  05:14 PM           827,840 tmdb.movies.csv.gz
09/30/2022  05:14 PM           153,218 tn.movie_budgets.csv.gz
               7 File(s)    174,378,326 bytes
               2 Dir(s)  34,863,439,872 bytes free


In [4]:
bottom_line = pd.read_csv("tn.movie_budgets.csv.gz")

In [5]:
bottom_line["production_budget"]
def clean_money(df, money_column):
    df[money_column] = df[money_column].str.replace(',','')
    df[money_column] = df[money_column].str.replace('$','')
    df[money_column] = pd.to_numeric(df[money_column])
    return df

#cleaning
bottom_line = clean_money(bottom_line,'production_budget')
bottom_line = clean_money(bottom_line,'domestic_gross')
bottom_line = clean_money(bottom_line,'worldwide_gross')

In [6]:
bottom_line["profit"] = bottom_line["worldwide_gross"] - bottom_line["production_budget"]
bottom_line

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350,-200237650
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963,1072413963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,999721747
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,7000,0,0,-7000
5778,79,"Apr 2, 1999",Following,6000,48482,240495,234495
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000,1338,1338,-3662
5780,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400


In [7]:
con = sqlite3.connect("./Ignore - im.db")
cursor = con.cursor()

In [8]:
pd.read_sql("""
SELECT *
FROM sqlite_master
""", con)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [19]:
persons = pd.read_sql("""
SELECT *
FROM persons
""", con)
persons

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,actress
606644,nm9990690,Joo Yeon So,NaN,NaN,actress
606645,nm9991320,Madeline Smith,NaN,NaN,actress
606646,nm9991786,Michelle Modigliani,NaN,NaN,producer


In [17]:
writers = pd.read_sql("""
SELECT *
FROM writers
""", con)
writers

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0438973,nm0175726
2,tt0438973,nm1802864
3,tt0462036,nm1940585
4,tt0835418,nm0310087
...,...,...
255868,tt8999892,nm10122246
255869,tt8999974,nm10122357
255870,tt9001390,nm6711477
255871,tt9004986,nm4993825


In [69]:
full_writers = writers.merge(persons, left_on="person_id", right_on="person_id", how="left")
full_writers.drop(labels=["person_id", "birth_year", "death_year", "primary_profession"], axis=1, inplace=True)
full_writers = full_writers.rename(columns={"primary_name": "writer_name"})

In [70]:
films = pd.read_sql("""
SELECT *
FROM movie_basics
""", con)
films

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [71]:
film_writers = full_writers.merge(films, left_on="movie_id", right_on="movie_id",
              how="left")
film_writers.drop(labels=["movie_id","genres", "runtime_minutes", "start_year", "original_title"], axis=1, inplace=True)
film_writers

,writer_name,primary_title
0,Tony Vitale,Life's a Beach
1,Steve Conrad,Sealand
2,Sean Sorensen,Sealand
3,Bill Haley,Steve Phoenix: The Untold Story
4,Peter Gaulke,The Babymakers
...,...,...
255868,Bradley T. Castle,Dumpster Fire: A Time Of Current Times
255869,Daysi Burbano,Madre Luna
255870,Bernard Lessa,The woman and the river
255871,Fredrik Horn Akselsen,Syndebukken: Prosessen mot Harry Lindstrøm


In [78]:
writer_profit = bottom_line.merge(film_writers, left_on="movie", right_on="primary_title", how="left")
writer_profit.dropna(subset=["writer_name"], inplace=True)
writer_profit

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit,writer_name,primary_title
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279,Yûsuke Yamada,Avatar
1,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279,Teruo Noguchi,Avatar
2,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,Ted Elliott,Pirates of the Caribbean: On Stranger Tides
3,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,Terry Rossio,Pirates of the Caribbean: On Stranger Tides
4,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,Stuart Beattie,Pirates of the Caribbean: On Stranger Tides
...,...,...,...,...,...,...,...,...,...
12017,73,"Jan 13, 2012",Newlyweds,9000,4584,4584,-4416,Edward Burns,Newlyweds
12022,78,"Dec 31, 2018",Red 11,7000,0,0,-7000,Racer Rodriguez,Red 11
12023,78,"Dec 31, 2018",Red 11,7000,0,0,-7000,Robert Rodriguez,Red 11
12026,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400,Benjamin Roberds,A Plague So Pleasant


In [93]:
prolific_writers = writer_profit.groupby('writer_name').filter(lambda x : len(x)>1)
prolific_writers

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,profit,writer_name,primary_title
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279,2351345279,Yûsuke Yamada,Avatar
2,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,Ted Elliott,Pirates of the Caribbean: On Stranger Tides
3,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,Terry Rossio,Pirates of the Caribbean: On Stranger Tides
4,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,Stuart Beattie,Pirates of the Caribbean: On Stranger Tides
5,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,635063875,Jay Wolpert,Pirates of the Caribbean: On Stranger Tides
...,...,...,...,...,...,...,...,...,...
12013,70,"Apr 1, 1996",Bang,10000,527,527,-9473,Zarina Tadjibaeva,Bang
12014,70,"Apr 1, 1996",Bang,10000,527,527,-9473,Zarina Tadjibaeva,Bang
12023,78,"Dec 31, 2018",Red 11,7000,0,0,-7000,Robert Rodriguez,Red 11
12026,81,"Sep 29, 2015",A Plague So Pleasant,1400,0,0,-1400,Benjamin Roberds,A Plague So Pleasant


In [95]:
prolific_writers["profit"].mean()

141447195.17632452